# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [2]:
# Загрузите данные из файла insurance.csv в переменную df

In [4]:
df=pd.read_csv('insurance.csv',sep=',')

In [5]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

## Преобразуем строковые данные

In [8]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1,'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [9]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [10]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False


## Формируем признаки и целевую переменную

In [11]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [12]:
## Сформируйте признаки и целевую переменную

In [13]:
x=df[['age', 'sex', 'bmi', 'children', 'smoker','region_northeast', 'region_northwest', 'region_southeast','region_southwest']]
y=df[['charges']]

In [14]:
x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,False,False,False,True
1,18,1,33.770,1,0,False,False,True,False
2,28,1,33.000,3,0,False,False,True,False
3,33,1,22.705,0,0,False,True,False,False
4,32,1,28.880,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,False,True,False,False
1334,18,0,31.920,0,0,True,False,False,False
1335,18,0,36.850,0,0,False,False,True,False
1336,21,0,25.800,0,0,False,False,False,True


In [15]:
y

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## Разделяем данные на выборку для обучения/проверки

In [16]:
x_train, x_test, y_train, y_test = train_test_split (x,y, test_size=0.2)

## Создаем и обучаем модель линейной регресии

In [17]:
lr=LinearRegression()

In [18]:
lr.fit(x_train, y_train)

LinearRegression()

In [19]:
lr.coef_

array([[ 2.54057471e+02, -3.67190213e+00,  3.22295387e+02,
         5.34383039e+02,  2.40410015e+04,  8.76734735e+02,
         1.01266467e+02, -5.68587791e+02, -4.09413410e+02]])

## Получаем предсказание и оцениваем качество

In [ ]:
lr.predict(x_test)

In [21]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [22]:
mean_squared_error(y_test, lr.predict(x_test))

29633801.454642322

In [23]:
mean_squared_error(y_train, lr.predict(x_train))

38300164.03486111

## Делаем предсказание для одного человека

In [24]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [25]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [26]:
lr.predict(df_person)

array([[27256.80668243]])